# Genotyping Pipeline Tutorial (IDAT → GTC → VCF → PLINK) — Bash-Only Notebook

Notebook ini dibuat untuk **siswa SMA** supaya tidak hanya “run-run”, tetapi benar-benar memahami:
- **input** yang dipakai,
- **command** yang dijalankan,
- **output** yang dihasilkan,
- dan **kenapa** tiap tahap diperlukan.

> Semua langkah dijalankan memakai **bash command** (via `%%bash`), jadi tampilannya mirip skrip pipeline.

---

## Prasyarat
Sebelum mulai, pastikan sudah tersedia:
- `iaap-cli` (sudah kamu bagikan)
- `bcftools`
- `plink`
- file chip: `.bpm`, `.csv`, `.egt`
- folder IDAT
- reference genome FASTA
- mapping `asa2rsid.map` dan `chr_map.txt`


## Aturan main praktikum
1. Jalankan cell **berurutan**.
2. Setelah tiap step, baca output validasi (`ls`, `head`, `wc`) dan jawab pertanyaannya.
3. Kalau error, stop dan pakai bagian **Troubleshooting** di bawah.


## Step 0 — Setup parameter (disimpan ke `params.sh`)
Karena setiap cell `%%bash` berjalan di shell baru, kita simpan semua parameter di file `params.sh` lalu kita `source` di setiap step.


In [ ]:
%%bash
set -euo pipefail
cat > params.sh <<'EOF'
# ===== INPUTS =====
MANIFEST_BPM="asa/ASA-24v1-0_E2.bpm"
MANIFEST_CSV="asa/ASA-24v1-0_E2.csv"
CLUSTER_EGT="asa/ASA-24v1-0_A1_ClusterFile.egt"
REF_FASTA="/scrap/icx/ref/38/human_v38.fasta"
IDAT_DIR="idat_test/idat"

ASA2RSID="snp_38/b150/asa2rsid.map"
CHRMAP="chr_map.txt"
REF_VERSION="38"

POP_DIR="pop"
POP="INA5"

# ===== OUTPUTS =====
OUT_DIR="gtc_genoplan_test"
THREADS=2
OUT_VCF_PREFIX="out_vcf"
OUT_PLINK_PREFIX="out_plink"
SAMPLE_ID="SAMPLE001"

# ===== DERIVED OUTPUT PATHS =====
OUT_GTC_DIR="${OUT_DIR}/gtc"
OUT_VCF_DIR="${OUT_DIR}/vcf/${REF_VERSION}"
OUT_PLINK_DIR="${OUT_DIR}/plink/${REF_VERSION}"
OUT_ALEL_DIR="${OUT_DIR}/allel/${REF_VERSION}"
EOF

echo "✅ params.sh dibuat. Silakan cek isinya:"
sed -n '1,120p' params.sh


### Checkpoint (Step 0)
- [ ] File `params.sh` sudah ada dan isinya sesuai folder/file yang kamu punya.
- [ ] `THREADS` sudah diset sesuai komputer (misal 2–4 untuk laptop).

**Pertanyaan**
1) Parameter mana yang termasuk **input data**?  
2) Parameter mana yang termasuk **output setting**?


## Step 0.5 — Buat folder output


In [ ]:
%%bash
set -euo pipefail
source params.sh
mkdir -p "$OUT_GTC_DIR" "$OUT_VCF_DIR" "$OUT_PLINK_DIR" "$OUT_ALEL_DIR"

echo "✅ Folder output dibuat:"
echo "  OUT_GTC_DIR  : $OUT_GTC_DIR"
echo "  OUT_VCF_DIR  : $OUT_VCF_DIR"
echo "  OUT_PLINK_DIR: $OUT_PLINK_DIR"
echo "  OUT_ALEL_DIR : $OUT_ALEL_DIR"

ls -lh "$OUT_DIR" || true


### Checkpoint (Step 0.5)
- [ ] Folder `gtc/`, `vcf/`, `plink/`, `allel/` sudah muncul di dalam `OUT_DIR`.

**Pertanyaan**
1) Kenapa kita pisahkan output per tahap (gtc/vcf/plink)?


## Step 0.9 — Cek tools (iaap-cli, bcftools, plink)


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "== TOOL CHECK =="
command -v iaap-cli && iaap-cli --help >/dev/null && echo "✅ iaap-cli OK" || echo "❌ iaap-cli belum ketemu di PATH"
command -v bcftools && bcftools --version | head -n 2 || echo "❌ bcftools belum ada"
command -v plink && plink --version | head -n 2 || echo "❌ plink belum ada"

echo
echo "== FILE CHECK (contoh) =="
ls -lh "$MANIFEST_BPM" "$MANIFEST_CSV" "$CLUSTER_EGT" 2>/dev/null || echo "⚠️ beberapa file chip belum ketemu (cek path di params.sh)"
ls -lh "$IDAT_DIR" 2>/dev/null | head || echo "⚠️ folder IDAT belum ketemu (cek path di params.sh)"


### Checkpoint (Step 0.9)
- [ ] `bcftools` dan `plink` terdeteksi.
- [ ] `iaap-cli` terdeteksi (kalau tidak, pastikan PATH sudah benar atau panggil pakai `./iaap-cli`).

**Pertanyaan**
1) Apa bedanya tool “bioinformatika” (bcftools/plink) dengan tool “vendor” (iaap-cli)?


## Step 1 — IDAT → GTC (Genotype Calling)

**Ide besar:**  
IDAT adalah data mentah intensitas dari scanner microarray. `iaap-cli gencall` mengubah intensitas menjadi “genotype calls” (mis. AA/AB/BB) dan menghasilkan file GTC.

### Output utama
- Folder `gtc/` berisi file `.gtc`


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP1] IDAT -> GTC"
echo "INPUT :"
echo "  IDAT_DIR     = $IDAT_DIR"
echo "  MANIFEST_BPM = $MANIFEST_BPM"
echo "  CLUSTER_EGT  = $CLUSTER_EGT"
echo "OUTPUT:"
echo "  OUT_GTC_DIR  = $OUT_GTC_DIR"
echo "THREADS: $THREADS"
echo

iaap-cli gencall "$MANIFEST_BPM" "$CLUSTER_EGT" "$OUT_GTC_DIR" \
  -f "$IDAT_DIR" -g -t "$THREADS"

echo
echo "✅ Validasi output (contoh 10 file pertama):"
ls -lh "$OUT_GTC_DIR" | head -n 10


### Checkpoint (Step 1)
- [ ] Ada file di `OUT_GTC_DIR` (biasanya `.gtc`).
- [ ] Kalau kosong: cek `IDAT_DIR` dan pastikan file IDAT lengkap.

**Pertanyaan**
1) Menurut kamu, “genotype calling” itu mengubah data seperti apa menjadi apa?  
2) Kenapa perlu `EGT (cluster file)`?


## Step 2 — GTC → VCF (standar varian)

**Ide besar:**  
VCF adalah format standar untuk menyimpan varian (chr, posisi, ref/alt) + genotype per sampel.

Kita juga melakukan:
- sort (urutkan varian)
- norm (normalisasi terhadap FASTA, buang duplikasi)
- index (buat akses cepat)

### Output utama
- `out_vcf.vcf.gz`
- `out_vcf.vcf.csi` (index)
- `out_vcf.tsv` (extra info dari konversi)


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP2] GTC -> VCF"
echo "INPUT :"
echo "  OUT_GTC_DIR  = $OUT_GTC_DIR"
echo "  MANIFEST_BPM = $MANIFEST_BPM"
echo "  MANIFEST_CSV = $MANIFEST_CSV"
echo "  CLUSTER_EGT  = $CLUSTER_EGT"
echo "  REF_FASTA    = $REF_FASTA"
echo "OUTPUT:"
echo "  VCF_GZ       = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz"
echo "  VCF_INDEX    = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.csi"
echo "  EXTRA_TSV    = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.tsv"
echo

bcftools +gtc2vcf -Oz --no-version \
  --bpm "$MANIFEST_BPM" \
  --csv "$MANIFEST_CSV" \
  --egt "$CLUSTER_EGT" \
  --gtcs "$OUT_GTC_DIR" \
  --fasta-ref "$REF_FASTA" \
  --extra "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.tsv" | \
  bcftools sort -Oz -T "${OUT_VCF_DIR}/bcftools-sort.XXXXXX" | \
  bcftools norm --no-version --rm-dup all -Oz -c x -f "$REF_FASTA" | \
  tee "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" | \
  bcftools index --force --output "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.csi"

echo
echo "✅ Validasi output:"
ls -lh "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.csi" "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.tsv" 2>/dev/null || true

echo
echo "✅ Preview header VCF (25 baris):"
bcftools view -h "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" | head -n 25


### Checkpoint (Step 2)
- [ ] File `.vcf.gz` dan `.vcf.csi` ada.
- [ ] Kamu bisa lihat header VCF muncul.

**Pertanyaan**
1) Kenapa VCF perlu di-**index**?  
2) Menurutmu fungsi `sort` dan `norm` apa?  
3) Di header VCF, cari baris yang berisi `##reference` atau info referensi. Ada nggak?


## Step 3 — VCF → PLINK (bed/bim/fam)

**Ide besar:**  
PLINK format cepat untuk analisis genetik. Kita ubah VCF jadi:
- `.bed` (binary genotype matrix)
- `.bim` (daftar varian)
- `.fam` (daftar sampel)

Di tahap ini juga:
- memecah multi-allelic → biallelic
- menormalkan lagi terhadap FASTA
- rename kromosom via `CHRMAP`
- memberi ID varian format `CHROM:POS:REF:ALT` supaya unik


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP3] VCF -> PLINK"
echo "INPUT :"
echo "  VCF_GZ   = ${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz"
echo "  REF_FASTA= $REF_FASTA"
echo "  CHRMAP   = $CHRMAP"
echo "OUTPUT:"
echo "  PLINK    = ${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.{bed,bim,fam}"
echo

bcftools norm -Ou -m -any "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz" | \
  bcftools norm -Ou -f "$REF_FASTA" | \
  bcftools annotate --rename-chrs "$CHRMAP" | \
  bcftools annotate -Ob -I '%CHROM:%POS:%REF:%ALT' | \
  plink --bcf /dev/stdin \
    --keep-allele-order \
    --vcf-idspace-to _ \
    --double-id \
    --allow-extra-chr 0 \
    --split-x b38 no-fail \
    --make-bed \
    --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}"

echo
echo "✅ Validasi output:"
ls -lh "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bed" "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bim" "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam" 2>/dev/null || true

echo
echo "✅ Preview .fam (5 baris):"
head -n 5 "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam"

echo
echo "✅ Preview .bim (5 baris):"
head -n 5 "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bim"


### Checkpoint (Step 3)
- [ ] File `.bed`, `.bim`, `.fam` ada.
- [ ] Kamu bisa lihat contoh isi `.fam` dan `.bim`.

**Pertanyaan**
1) Dari `.fam`, kolom mana yang terlihat seperti “ID sampel”?  
2) Dari `.bim`, kolom mana yang terlihat seperti “posisi varian”?  
3) Kenapa kita pilih ID varian `CHROM:POS:REF:ALT` (bukan rsID dulu)?


## Step 4 — Update Sample ID

**Ide besar:**  
Kadang ID sampel dari proses konversi bukan nama yang kita inginkan. Kita ganti supaya rapih dan konsisten.

Output yang dicek:
- `.fam` harus menunjukkan `SAMPLE_ID` yang kamu set.


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP4] Update Sample ID"
echo "TARGET SAMPLE_ID: $SAMPLE_ID"

IDAT_ID=$(awk '{print $1; exit}' "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam")

echo "IDAT_ID (lama)  : $IDAT_ID"
echo "SAMPLE_ID (baru): $SAMPLE_ID"

echo -e "${IDAT_ID}\t${IDAT_ID}\t${SAMPLE_ID}\t${SAMPLE_ID}" > "${OUT_PLINK_DIR}/sample.map"

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}" \
  --update-ids "${OUT_PLINK_DIR}/sample.map" \
  --make-bed --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}"

echo
echo "✅ Preview .fam setelah update (5 baris):"
head -n 5 "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.fam"


### Checkpoint (Step 4)
- [ ] `.fam` sekarang menampilkan ID yang kamu harapkan.

**Pertanyaan**
1) Menurutmu kenapa “penamaan sampel” penting dalam pipeline bioinformatika?  
2) Apa risiko kalau nama sampel berubah-ubah di setiap tahap?


## Step 5 — Sex Check (QC dasar pakai chr X/Y)

**Ide besar:**  
Kita ambil kromosom 23–24 (X dan Y), lalu menjalankan `--check-sex`.
Ini membandingkan:
- sex yang dicatat di `.fam` (atau update-sex)
vs
- sex yang diprediksi dari pola genotipe X/Y

Output:
- file `sex/sample.sexcheck`


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP5] Sex check"
OUTDIR_SEX="${OUT_DIR}/sex"
mkdir -p "$OUTDIR_SEX"

echo "OUTPUT: ${OUTDIR_SEX}/sample.sexcheck"
echo

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}" \
  --chr 23-24 --make-bed \
  --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}_2324"

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}_2324" \
  --update-name "$ASA2RSID" --make-bed \
  --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}_2324_upd"

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}_2324_upd" \
  --check-sex \
  --read-freq "${POP_DIR}/${POP}/${POP}_38_freq.frq" \
  --out "${OUTDIR_SEX}/sample"

echo
echo "✅ Preview sexcheck (header + 10 baris):"
head -n 12 "${OUTDIR_SEX}/sample.sexcheck"


### Checkpoint (Step 5)
- [ ] `sample.sexcheck` ada dan bisa dibaca.
- [ ] Kamu melihat kolom-kolom seperti `STATUS` atau nilai terkait F-stat/X.

**Pertanyaan**
1) Kenapa sex check fokus ke kromosom X/Y?  
2) Menurutmu apa arti “mismatch sex” dan kenapa ini penting?

**Mini-Challenge**
- Cari di output: ada label seperti `OK` atau `PROBLEM`? Jelaskan artinya.


## Step 6 — Membuat `variant.list` (ringkasan varian + genotype)

**Ide besar:**  
Dari VCF, kita buat list sederhana:
- ID varian (di-map ke rsID jika ada di `ASA2RSID`)
- genotype (mis. 0/1, 1/1, dll)

Output:
- `allel/<ref_version>/variant.list`

> Catatan: di sini kita memakai `process substitution <( ... )`, jadi kita jalankan dalam bash.


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP6] Create variant.list"
echo "OUTPUT: ${OUT_ALEL_DIR}/variant.list"
echo

awk -F'\t' 'NR==FNR {a[$1]=$2; next} { if($1 in a) {print a[$1]"\t"$2} else {print $0} }' "$ASA2RSID" \
  <(awk -F'\t' '/^[^#]/{split($10,a,":"); gsub(/^chr/, "", $1); print $1":"$2":"$4":"$5"\t"a[1]}' \
    <(gunzip -c "${OUT_VCF_DIR}/${OUT_VCF_PREFIX}.vcf.gz")) \
  > "${OUT_ALEL_DIR}/variant.list"

echo "✅ Preview variant.list (10 baris):"
head -n 10 "${OUT_ALEL_DIR}/variant.list"

echo
echo "✅ Hitung jumlah baris:"
wc -l "${OUT_ALEL_DIR}/variant.list"


### Checkpoint (Step 6)
- [ ] `variant.list` berhasil dibuat.
- [ ] Kamu bisa melihat pasangan “ID → genotype”.

**Pertanyaan**
1) Apa keuntungan membuat `variant.list` dibanding membaca VCF langsung?  
2) Kalau genotype `0/0`, `0/1`, `1/1` artinya apa?

**Mini-Challenge**
- Cari 1 baris `variant.list` dan jelaskan artinya dalam kalimat sederhana.


## Step 7 — Update SNP ID ke rsID + export `out23.txt`

**Ide besar:**  
- `--update-name` mengganti ID varian di PLINK menggunakan mapping `ASA2RSID` (jika tersedia).
- `out23.txt` adalah tabel ringkas ala “genotype report” (bukan format 23andMe resmi).

Output:
- `vcf/<ref_version>/out23.txt`


In [ ]:
%%bash
set -euo pipefail
source params.sh

echo "[STEP7] Update SNPID + Export out23.txt"
echo "OUTPUT: ${OUT_VCF_DIR}/out23.txt"
echo

plink --bfile "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}" \
  --update-name "$ASA2RSID" --make-bed \
  --out "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}"

awk -F'\t' '{split($2,a,":"); print a[1]"\t"$1"\t"$4"\t"$6$5}' \
  "${OUT_PLINK_DIR}/${OUT_PLINK_PREFIX}.bim" > "${OUT_VCF_DIR}/out23.txt"

echo "✅ Preview out23.txt (10 baris):"
head -n 10 "${OUT_VCF_DIR}/out23.txt"


### Checkpoint (Step 7)
- [ ] `out23.txt` ada dan bisa dibaca.
- [ ] Kolom-kolomnya terlihat seperti: rsID, chromosome, position, genotype.

**Pertanyaan**
1) Kenapa kadang rsID bisa bermasalah (duplicate/ambigu) sehingga pipeline sebelumnya pakai `CHROM:POS:REF:ALT`?  
2) Apa bedanya “ID varian” vs “posisi varian”?

**Mini-Project (opsional)**
- Ambil 5 baris `out23.txt` dan buat “mini laporan”:
  - varian apa,
  - berada di kromosom berapa,
  - genotipe apa.


# Troubleshooting cepat

Kalau ada error, cek hal berikut (jalankan cell sesuai kebutuhan):
- path file salah (`params.sh`)
- tool belum ada di PATH
- permission (`chmod +x iaap-cli`)
- file IDAT kurang lengkap
- referensi FASTA beda build (GRCh37 vs GRCh38)


In [ ]:
%%bash
set -euo pipefail
set -euo pipefail
echo "== QUICK CHECK: current dir & files =="
pwd
ls -lh | head -n 30

echo
echo "== QUICK CHECK: params.sh =="
test -f params.sh && echo "✅ params.sh exists" || echo "❌ params.sh missing"


In [ ]:
%%bash
set -euo pipefail
set -euo pipefail
source params.sh

echo "== TOOL PATH =="
command -v iaap-cli || true
command -v bcftools || true
command -v plink || true

echo
echo "== INPUT FILES EXIST? =="
for f in "$MANIFEST_BPM" "$MANIFEST_CSV" "$CLUSTER_EGT" "$ASA2RSID" "$CHRMAP"; do
  if [ -f "$f" ]; then echo "✅ $f"; else echo "❌ missing: $f"; fi
done

echo
echo "== IDAT DIR EXIST? =="
if [ -d "$IDAT_DIR" ]; then
  echo "✅ $IDAT_DIR"
  ls -lh "$IDAT_DIR" | head
else
  echo "❌ missing dir: $IDAT_DIR"
fi

echo
echo "== OUTPUT FILES (if already ran some steps) =="
ls -lh "$OUT_GTC_DIR" 2>/dev/null | head || true
ls -lh "$OUT_VCF_DIR" 2>/dev/null | head || true
ls -lh "$OUT_PLINK_DIR" 2>/dev/null | head || true


# Catatan untuk pengajar
Kalau kamu ingin kelas lebih “aktif”, kamu bisa:
- ganti `SAMPLE_ID` per siswa (mis. pakai nama kelompok)
- beri tugas: “ubah THREADS, bandingkan runtime”
- beri tugas: “ubah OUT_DIR, jelaskan kenapa reproducibility penting”
